In [1]:
# %% General imports
import numpy as np
import torch

# DeepMoD stuff
from multitaskpinn import DeepMoD
from multitaskpinn.model.func_approx import NN
from multitaskpinn.model.library import Library1D
from multitaskpinn.model.constraint import LeastSquares
from multitaskpinn.model.sparse_estimators import Threshold
from multitaskpinn.training.sparsity_scheduler import TrainTestPeriodic
from multitaskpinn.training import train_bayes, train_SBL

from multitaskpinn.data import Dataset
from multitaskpinn.data.kdv import DoubleSoliton

%load_ext autoreload
%autoreload 2

In [2]:
# Settings
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device='cpu'
# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [3]:
# %% making data

x = np.linspace(-10, 10, 50)
t = np.linspace(0.0, 2.0, 20)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')

dataset = Dataset(DoubleSoliton, c=[5.0, 1.0], x0=[-5.0, -1.0])
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=0.20, random=True, normalize=True)
X, y = X.to(device), y.to(device)


# Bayes

In [8]:
sparsity_scheduler = TrainTestPeriodic(periodicity=25, patience=200, delta=1e-5) # in terms of write iterations
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint, 1).to(device) # Putting it all in the model
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True, lr=2e-3) # Defining optimizer

In [9]:
train_bayes(model, X, y, optimizer, sparsity_scheduler, exp_ID=f'bayes_deepmod_thresholded_0.1_low_d', split=0.8, write_iterations=50, max_iterations=100000, delta=1e-5, patience=200)

 10600  MSE: 1.95e-02  Reg: 9.26e-08  L1: 1.72e+00 

KeyboardInterrupt: 

# Bayes pinn

In [21]:
sparsity_scheduler = TrainTestPeriodic(periodicity=25, patience=1e8, delta=1e-5) # in terms of write iterations
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint, 1).to(device) # Putting it all in the model
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True, lr=1e-3) # Defining optimizer

In [22]:
model.constraint.sparsity_masks = [torch.zeros(12, dtype=torch.bool).to(device)]
model.constraint.sparsity_masks[0][[3, 5]] = 1

In [ ]:
train_bayes(model, X, y, optimizer, sparsity_scheduler, exp_ID=f'same_noise', split=0.8, write_iterations=50, max_iterations=100000, delta=0.0, patience=200)

 96250  MSE: 1.02e-02  Reg: 7.29e-05  L1: 2.49e+00 

# SBL

In [7]:
sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=1e8, delta=0.0) # in terms of write iterations
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.0) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint, 12).to(device) # Putting it all in the model
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True, lr=2e-3) # Defining optimizer

In [8]:
train_SBL(model, X, y, optimizer, sparsity_scheduler, exp_ID=f'SBL_2e-3', split=0.8, write_iterations=50, max_iterations=100000, delta=0.0, patience=200)

 98150  MSE: 1.41e+00  Reg: 5.72e-10  L1: 1.00e+00 

KeyboardInterrupt: 

# SBL two optimzers

In [36]:
sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=1e8, delta=0.0) # in terms of write iterations
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.0) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint, 12).to(device) # Putting it all in the model
optimizer = torch.optim.Adam([{'params':model.func_approx.parameters(), 'betas':(0.99, 0.999), 'amsgrad':True, 'lr':2e-3},
                              {'params':[model.b, model.t], 'betas':(0.99, 0.999), 'amsgrad':True, 'lr':2e-3}, 
                              {'params':[model.a], 'betas':(0.99, 0.999), 'amsgrad':True, 'lr':2e-3}]) # Defining optimizer

In [37]:
train_SBL(model, X, y, optimizer, sparsity_scheduler, exp_ID=f'SBL_diff_noise_even', split=0.8, write_iterations=50, max_iterations=50000, delta=0.0, patience=200)

 21650  MSE: 1.96e+00  Reg: 9.71e-09  L1: 1.00e+00 

KeyboardInterrupt: 

# SBL PINN

In [52]:
sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=1e8, delta=0.0) # in terms of write iterations
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.0) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint, 12).to(device) # Putting it all in the model
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True, lr=2e-3) # Defining optimizer

In [53]:
model.constraint.sparsity_masks = [torch.zeros(12, dtype=torch.bool).to(device)]
model.constraint.sparsity_masks[0][[3, 5]] = 1

In [54]:
model.constraint.sparsity_masks

[tensor([False, False, False,  True, False,  True, False, False, False, False,
         False, False])]

In [55]:
train_SBL(model, X, y, optimizer, sparsity_scheduler, exp_ID=f'SBL_PINN_2e3', split=0.8, write_iterations=50, max_iterations=50000, delta=0.0, patience=200)

 49950  MSE: 3.52e-02  Reg: 7.42e-01  L1: 1.09e+00 Algorithm converged. Writing model to disk.
